In [3]:
import sys
sys.path.append('../')

import pandas as pd

import glob
# from dim_reduction import umap_projection, plot_dm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [4]:
import torch
import numpy as np
import nibabel as nib
from utils import cropVolume
import Model as Net
import os

In [5]:
1

1

In [6]:
folders = glob.glob("../data/test/*/*")
folders



['../data/test/LGG/Brats18_TCIA09_462_1',
 '../data/test/LGG/Brats18_TCIA09_620_1',
 '../data/test/LGG/Brats18_TCIA09_141_1',
 '../data/test/LGG/Brats18_TCIA13_633_1',
 '../data/test/LGG/Brats18_TCIA10_310_1',
 '../data/test/LGG/Brats18_TCIA13_645_1',
 '../data/test/LGG/Brats18_TCIA10_644_1',
 '../data/test/LGG/Brats18_TCIA10_202_1',
 '../data/test/LGG/Brats18_TCIA13_642_1',
 '../data/test/LGG/Brats18_2013_24_1',
 '../data/test/LGG/Brats18_TCIA09_177_1',
 '../data/test/LGG/Brats18_TCIA10_266_1',
 '../data/test/LGG/Brats18_TCIA13_624_1',
 '../data/test/LGG/Brats18_TCIA09_451_1',
 '../data/test/LGG/Brats18_TCIA10_629_1',
 '../data/test/LGG/Brats18_TCIA10_632_1',
 '../data/test/LGG/Brats18_TCIA10_241_1',
 '../data/test/LGG/Brats18_TCIA10_130_1',
 '../data/test/LGG/Brats18_TCIA13_653_1',
 '../data/test/LGG/Brats18_2013_0_1',
 '../data/test/LGG/Brats18_TCIA10_639_1',
 '../data/test/LGG/Brats18_TCIA10_282_1',
 '../data/test/LGG/Brats18_TCIA13_623_1',
 '../data/test/LGG/Brats18_TCIA10_307_1',

In [15]:
'/'.join(folders[1].split('/')[-2:])

'LGG/Brats18_TCIA09_620_1'

In [18]:
['/'.join(folder.split('/')[-2:]) for folder in folders]

['LGG/Brats18_TCIA09_462_1',
 'LGG/Brats18_TCIA09_620_1',
 'LGG/Brats18_TCIA09_141_1',
 'LGG/Brats18_TCIA13_633_1',
 'LGG/Brats18_TCIA10_310_1',
 'LGG/Brats18_TCIA13_645_1',
 'LGG/Brats18_TCIA10_644_1',
 'LGG/Brats18_TCIA10_202_1',
 'LGG/Brats18_TCIA13_642_1',
 'LGG/Brats18_2013_24_1',
 'LGG/Brats18_TCIA09_177_1',
 'LGG/Brats18_TCIA10_266_1',
 'LGG/Brats18_TCIA13_624_1',
 'LGG/Brats18_TCIA09_451_1',
 'LGG/Brats18_TCIA10_629_1',
 'LGG/Brats18_TCIA10_632_1',
 'LGG/Brats18_TCIA10_241_1',
 'LGG/Brats18_TCIA10_130_1',
 'LGG/Brats18_TCIA13_653_1',
 'LGG/Brats18_2013_0_1',
 'LGG/Brats18_TCIA10_639_1',
 'LGG/Brats18_TCIA10_282_1',
 'LGG/Brats18_TCIA13_623_1',
 'LGG/Brats18_TCIA10_307_1',
 'LGG/Brats18_TCIA13_654_1',
 'LGG/Brats18_TCIA13_621_1',
 'LGG/Brats18_2013_8_1',
 'LGG/Brats18_TCIA10_387_1',
 'LGG/Brats18_2013_16_1',
 'LGG/Brats18_TCIA10_420_1',
 'LGG/Brats18_TCIA09_493_1',
 'LGG/Brats18_TCIA10_175_1',
 'LGG/Brats18_TCIA10_152_1',
 'LGG/Brats18_TCIA09_312_1',
 'LGG/Brats18_TCIA10_637_1',

In [55]:
folders = glob.glob("../data/test/*/*")
writeText = open('../data/test/test.txt', 'w')
for folder in folders:
#     folder = str('/'.join(folder.split('/')[-2:]) + '/')
#     print(folder)
    files = glob.glob(folder + '/*_flair.nii.gz')

    if len(files) <= 0:
        continue
    path = os.sep + files[0] + ', ' + '\n'
#     print('/'.join(path.split('/')[-3:]))
    save_path = '/'.join(path.split('/')[-3:])
    print(save_path)
    writeText.write(save_path)
writeText.close()

LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_flair.nii.gz, 

LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_flair.nii.gz, 

LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_flair.nii.gz, 

LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_flair.nii.gz, 

LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_flair.nii.gz, 

LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_flair.nii.gz, 

LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_flair.nii.gz, 

LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_flair.nii.gz, 

LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_flair.nii.gz, 

LGG/Brats18_2013_24_1/Brats18_2013_24_1_flair.nii.gz, 

LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_flair.nii.gz, 

LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_flair.nii.gz, 

LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_flair.nii.gz, 

LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_flair.nii.gz, 

LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_flair.nii.gz, 

LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_flair.nii.gz, 

LGG/Brats18_TC

In [56]:
!cat ../data/test/test.txt

LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_flair.nii.gz, 
LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_flair.nii.gz, 
LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_flair.nii.gz, 
LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_flair.nii.gz, 
LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_flair.nii.gz, 
LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_flair.nii.gz, 
LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_flair.nii.gz, 
LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_flair.nii.gz, 
LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_flair.nii.gz, 
LGG/Brats18_2013_24_1/Brats18_2013_24_1_flair.nii.gz, 
LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_flair.nii.gz, 
LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_flair.nii.gz, 
LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_flair.nii.gz, 
LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_flair.nii.gz, 
LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_flair.nii.gz, 
LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_flair.nii.gz, 
LGG/Brats18_TCIA10_241_1/Brats

In [77]:
def norm(im):
    '''
    Normalize the image using min-max normalization
    :param im: Input image
    :return: Min-max normalized image
    '''
    im = im.astype(np.float32)

    # min-max normalization [0,1]
    min_v = np.min(im)
    max_v = np.max(im)
    im = (im - min_v) / (max_v - min_v)
    return im

def cropVolumes(img1, img2, img3, img4):
    '''
    This function crop the 4 volumes that BRATS dataset provides
    :param img1: Volume 1
    :param img2: Volume 2
    :param img3: Volume 3
    :param img4: Volume 4
    :return: maximum dimensions across three dimensions
    '''
    ch_s1, ch_e1, wi_s1, wi_e1, hi_s1, hi_e1 = cropVolume(img1, True)
    ch_s2, ch_e2, wi_s2, wi_e2, hi_s2, hi_e2 = cropVolume(img2, True)
    ch_s3, ch_e3, wi_s3, wi_e3, hi_s3, hi_e3 = cropVolume(img3, True)
    ch_s4, ch_e4, wi_s4, wi_e4, hi_s4, hi_e4 = cropVolume(img4, True)

    # find the maximum dimensions
    ch_st = min(ch_s1, ch_s2, ch_s3, ch_s4)
    ch_en = max(ch_e1, ch_e2, ch_e3, ch_e4)
    wi_st = min(wi_s1, wi_s2, wi_s3, wi_s4)
    wi_en = max(wi_e1, wi_e2, wi_e3, wi_e4)
    hi_st = min(hi_s1, hi_s2, hi_s3, hi_s4)
    hi_en = max(hi_e1, hi_e2, hi_e3, hi_e4)

    return wi_st, wi_en, hi_st, hi_en, ch_st, ch_en


def segmentVoxel(imgLoc, model, device='cpu', out_dir='./predictions'):
    '''
    Segment the image and store the results
    :param imgLoc: location of the flair image. Other modalities locations can be computed using this one.
    :param model: Segmentation Network
    :return: dice scores
    '''
    t1_loc = imgLoc.replace('flair', 't1')
    t1ce_loc = imgLoc.replace('flair', 't1ce')
    t2_loc = imgLoc.replace('flair', 't2')
    label_loc = imgLoc.replace('flair', 'seg')

    # get the 4 modalities
    img_flair = nib.load(imgLoc).get_data()
    gth_dummy = np.copy(img_flair)
    img_t1 = nib.load(t1_loc).get_data()
    img_t1ce = nib.load(t1ce_loc).get_data()
    img_t2 = nib.load(t2_loc).get_data()

    # Crop and min-max normalize them
    wi_st, wi_en, hi_st, hi_en, ch_st, ch_en = cropVolumes(img_flair, img_t1, img_t1ce, img_t2)
    img_flair = norm(img_flair[wi_st:wi_en, hi_st:hi_en, ch_st:ch_en])
    img_t1 = norm(img_t1[wi_st:wi_en, hi_st:hi_en, ch_st:ch_en])
    img_t1ce = norm(img_t1ce[wi_st:wi_en, hi_st:hi_en, ch_st:ch_en])
    img_t2 = norm(img_t2[wi_st:wi_en, hi_st:hi_en, ch_st:ch_en])


    # convert to Tensor
    resize = (1, img_flair.shape[0], img_flair.shape[1], img_flair.shape[2])
    img_flair = img_flair.reshape(resize)
    img_t1 = img_t1.reshape(resize)
    img_t1ce = img_t1ce.reshape(resize)
    img_t2 = img_t2.reshape(resize)

    tensor_flair = torch.from_numpy(img_flair)
    tensor_t1 = torch.from_numpy(img_t1)
    inputB = torch.from_numpy(img_t1ce)
    tensor_t2 = torch.from_numpy(img_t2)
    del img_flair, img_t1, img_t1ce, img_t2

    # concat the tensors and then feed them to the model
    tensor_concat = torch.cat([tensor_flair, tensor_t1, inputB, tensor_t2], 0)  # inputB #
    tensor_concat = torch.unsqueeze(tensor_concat, 0)
    tensor_concat = tensor_concat.to(device)
    print(device)

    #convert to variable.
    # If you are using PyTorch version > 0.3, then you don't need this
#     tensor_concat_var = torch.autograd.Variable(tensor_concat, volatile=True) ###

    # Average ensembling at multiple resolutions
    # We found by experiments that ensembling at original and 144x144x144 gives best results.
    test_resolutions = [None, (144, 144, 144)]
    output = None
    for res in test_resolutions:
        if output is not None:
            # test at the scaled resolution and combine them
            output = output + model(tensor_concat, inp_res=res)
        else:
            # test at the original resolution
            output = model(tensor_concat, inp_res=res)
    output = output / len(test_resolutions)
    del tensor_concat

    # convert the output to segmentation mask and move to CPU
    output_ = output[0].max(0)[1].data.byte().cpu().numpy()


    output_numpy = np.zeros_like(gth_dummy)
    output_numpy[wi_st:wi_en, hi_st:hi_en, ch_st:ch_en] = output_

    # rename the label 3 back to label 4
    output_numpy[output_numpy == 3] = 4

    # We need headers, so reload the flair image again
    img1 = nib.load(imgLoc)
    img1_new = nib.Nifti1Image(output_numpy, img1.affine, img1.header)
    name = imgLoc.replace('_flair', '_seg').split('/')[-1]
    
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
    file_name = out_dir + os.sep + name
    print(file_name)
    nib.save(img1_new, file_name)

In [21]:
data_dir = '../data/test/' # evaluate on original data and not the processed one
test_file = 'test.txt'
if not os.path.isfile(data_dir + os.sep + test_file):
    print('Validation file not found')
#     exit(-1)

In [22]:
best_model_loc = '../pretrained/espnet_3d_brats.pth'
if not os.path.isfile(best_model_loc):
    print('Pretrained weight file does not exist. Please check')
#     exit(-1)

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net.ESPNet(classes=4, channels=4)
model.load_state_dict(torch.load(best_model_loc, map_location=device))
model.to(device)

ESPNet(
  (input1): InputProjectionA(
    (pool): ModuleList(
      (0): AvgPool3d(kernel_size=3, stride=2, padding=1)
    )
  )
  (input2): InputProjectionA(
    (pool): ModuleList(
      (0): AvgPool3d(kernel_size=3, stride=2, padding=1)
    )
  )
  (level0): CBR(
    (conv): Conv3d(4, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn): BatchNorm3d(16, eps=0.001, momentum=0.95, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
  )
  (level1): ModuleList(
    (0): DilatedParllelResidualBlockB1(
      (c1): CBR(
        (conv): Conv3d(16, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn): BatchNorm3d(8, eps=0.001, momentum=0.95, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (d1): CDilated(
        (conv): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (d2): CDilated(
        (conv): Conv3d(8, 8, kernel_size=(3, 3, 3), strid

In [44]:
device

device(type='cuda', index=0)

In [25]:
dice_scores_wt = []
dice_scores_cm = []
dice_scores_et = []

In [58]:
with open(data_dir + test_file) as txtFile:
    print(txtFile)
    for line in txtFile:
        line_arr = line.split(',')
        img_file = ((data_dir).strip() + '/' + line_arr[0].strip()).strip()
        print(img_file)
#         segmentVoxel(img_file, model, device)

<_io.TextIOWrapper name='../data/test/test.txt' mode='r' encoding='UTF-8'>
../data/test//LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_flair.nii.gz
../data/test//LGG/Brats18_2013_24_1/Brats18_2013_24_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_flair.nii.gz
../data/test//LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_6

In [41]:
data_dir + test_file

'../data/test/test.txt'

In [ ]:
out_dir='./predictions'

In [78]:
from tqdm import tqdm_notebook as tqdm
out_dir = '../../miccai2020/data/predictions'
flair_files = [x for x in glob.glob('../../miccai2020/data/mr_data/without_seg/*/*') if 'flair' in x]
for img_file in tqdm(flair_files):
    print(img_file)
    segmentVoxel(img_file, model, device=device, out_dir=out_dir)

../../miccai2020/data/mr_data/without_seg/Brats18_TCIA13_649_1/Brats18_TCIA13_649_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA13_649_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA04_253_1/Brats18_TCIA04_253_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA04_253_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA10_232_1/Brats18_TCIA10_232_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA10_232_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA13_626_1/Brats18_TCIA13_626_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA13_626_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA01_215_1/Brats18_TCIA01_215_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA01_215_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA02_139_1/Brats18_TCIA02_139_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats1

cuda:0
../../miccai2020/data/predictions//Brats18_TCIA03_250_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA02_355_1/Brats18_TCIA02_355_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA02_355_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA13_619_1/Brats18_TCIA13_619_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA13_619_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA13_617_1/Brats18_TCIA13_617_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA13_617_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA02_457_1/Brats18_TCIA02_457_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA02_457_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA10_195_1/Brats18_TCIA10_195_1_flair.nii.gz
cuda:0
../../miccai2020/data/predictions//Brats18_TCIA10_195_1_seg.nii.gz
../../miccai2020/data/mr_data/without_seg/Brats18_TCIA13_638_1/Brats18_